In [9]:
%logstop
%logstart -rtq ~/.logs/dw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [4]:
from static_grader import grader

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [5]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
File ‘./dw-data/201701scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/practices.csv.gz’ already there; not retrieving.

File ‘./dw-data/chem.csv.gz’ already there; not retrieving.



## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [6]:
import pandas as pd
import numpy as np
import gzip

In [7]:
# load the 2017 data
with gzip.open('./dw-data/201701scripts_sample.csv.gz', 'rb') as f:
    scripts = pd.read_csv(f)

print('Shape of scripts', scripts.shape)
scripts.head()

Shape of scripts (973193, 7)


,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [8]:
with gzip.open('./dw-data/practices.csv.gz', 'rb') as f:
    practices_0 = pd.read_csv(f)
    
print('Shape of practices:', practices_0.shape)
practices_0.head()

Shape of practices: (12019, 7)


,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
0,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
1,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
2,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
3,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ
4,A81006,TENNANT STREET MEDICAL PRACTICE,TENNANT ST MED PRACT,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AT


In [9]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices = practices_0.rename({practices_0.columns[i]: col_names[i] for i in np.arange(len(col_names))}, axis = 1)
print(practices.shape)
practices.head()

(12019, 7)


,code,name,addr_1,addr_2,borough,village,post_code
0,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
1,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
2,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
3,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ
4,A81006,TENNANT STREET MEDICAL PRACTICE,TENNANT ST MED PRACT,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AT


In [10]:
with gzip.open('./dw-data/chem.csv.gz', 'rb') as f:
    chem = pd.read_csv(f)
print('Shape of chem: ', chem.shape)
chem.head()

Shape of chem:  (3487, 2)


,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [11]:
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [12]:
scripts['items'].median()

2.0

In [13]:
def describe(df, key):
    total = df[key].sum()
    mean = df[key].mean()
    s = df[key].std()
    q25 = df[key].quantile(0.25)
    med = df[key].median()
    q75 = df[key].quantile(0.75)
    
    return (total, mean, s, q25, med, q75)

In [14]:
summary_stats = [('items', describe(scripts,'items')), ('quantity', describe(scripts,'quantity')), ('nic', describe(scripts,'nic')), ('act_cost', describe(scripts,'act_cost'))]

In [15]:
grader.score.dw__summary_statistics(summary_stats)

Your score:  1.0


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [16]:
sum_items = scripts.groupby('bnf_name')['items'].sum()
print(sum_items)

bnf_name
365 Film 10cm x 12cm VP Adh Film Dress       2
365 Non Adherent 10cm x 10cm Pfa Plas Fa     3
365 Non Adherent 10cm x 20cm Pfa Plas Fa     1
365 Non Woven Island 8cm x 10cm Adh Dres     1
365 Transpt Island 5cm x 7.2cm VP Adh Fi     2
                                            ..
nSpire PiKo-1 Stnd Range Peak Flow Meter     1
nSpire Pocket Peak Low Range Peak Flow M     4
nSpire Pocket Peak Stnd Range Peak Flow      8
oraNurse_Toothpaste Orig (1450ppm)           4
palmdoc (Reagent)_Strips                    59
Name: items, Length: 13471, dtype: int64


In [17]:
most_common_item = [(sum_items.idxmax(), sum_items.max())]

In [18]:
grader.score.dw__most_common_item(most_common_item)

Your score:  1.0


## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [19]:
# change the 'practice' column name to 'code'
scripts = scripts.rename({'practice': 'code'}, axis = 1)
scripts.head()

,code,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [20]:
# merge two data frames: "scripts" and "practices"
merged_df = pd.merge(scripts, practices, on = ['code'])

In [21]:
print(merged_df.shape)
merged_df.head()

(1134693, 13)


,code,bnf_code,bnf_name,items,nic,act_cost,quantity,name,addr_1,addr_2,borough,village,post_code
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF


In [22]:
# Group by postal code and bnf_name and calculate the sum of items: 
sum_items = merged_df.groupby(['post_code', 'bnf_name'])['items'].sum()

In [23]:
sum_items_df = sum_items.to_frame()
sum_items_df.head()

items
post_code bnf_name                                       
B11 4BW   3m Health Care_Cavilon Durable Barrier C      7
          3m Health Care_Cavilon No Sting Barrier       2
          Abasaglar KwikPen_100u/ml 3ml Pf Pen          2
          Abidec_Dps                                   63
          Able Spacer + Sml/Med Mask                    1

In [24]:
# Calculate the total of items for each postal code: 
total_items_per_postcode = sum_items.groupby(level = 'post_code').sum()

In [25]:
total_items_per_postcode

post_code
B11 4BW     22731
B12 9LP     23236
B18 7AL     20508
B21 9RY     31027
B23 6DJ     28011
            ...  
WS7 0EW     14122
WS9 8AJ     24483
WV13 2DR    36381
YO11 1UB    28507
YO16 4LZ    53847
Name: items, Length: 289, dtype: int64

In [26]:
type(total_items_per_postcode)

pandas.core.series.Series

In [27]:
total_items_per_postcode_df = total_items_per_postcode.to_frame()

In [28]:
total_items_per_postcode_df = total_items_per_postcode_df.rename({'items': 'total per post code'}, axis = 1)
total_items_per_postcode_df.head()

,total per post code
post_code,
B11 4BW,22731
B12 9LP,23236
B18 7AL,20508
B21 9RY,31027
B23 6DJ,28011


In [29]:
# merge the dataframe 'total_items_per_postcode_df' to 'sum_items_df'
sum_items_df_merge = sum_items_df.merge(total_items_per_postcode_df, on = ['post_code'], how = 'left')

In [30]:
sum_items_df_merge.head()

,items,total per post code
post_code,,
B11 4BW,7,22731
B11 4BW,2,22731
B11 4BW,2,22731
B11 4BW,63,22731
B11 4BW,1,22731


In [31]:
# add column 'total per post code' to the dataframe "sum_items_df"
sum_items_df['total per post code'] = list(sum_items_df_merge['total per post code'])

In [32]:
sum_items_df.head()

items  total per post code
post_code bnf_name                                                            
B11 4BW   3m Health Care_Cavilon Durable Barrier C      7                22731
          3m Health Care_Cavilon No Sting Barrier       2                22731
          Abasaglar KwikPen_100u/ml 3ml Pf Pen          2                22731
          Abidec_Dps                                   63                22731
          Able Spacer + Sml/Med Mask                    1                22731

In [33]:
# calculate the ratio of items by 'bnf_name'
sum_items_df['ratio of items'] = sum_items_df['items']/sum_items_df['total per post code']

In [34]:
sum_items_df.head()

items  \
post_code bnf_name                                          
B11 4BW   3m Health Care_Cavilon Durable Barrier C      7   
          3m Health Care_Cavilon No Sting Barrier       2   
          Abasaglar KwikPen_100u/ml 3ml Pf Pen          2   
          Abidec_Dps                                   63   
          Able Spacer + Sml/Med Mask                    1   

                                                    total per post code  \
post_code bnf_name                                                        
B11 4BW   3m Health Care_Cavilon Durable Barrier C                22731   
          3m Health Care_Cavilon No Sting Barrier                 22731   
          Abasaglar KwikPen_100u/ml 3ml Pf Pen                    22731   
          Abidec_Dps                                              22731   
          Able Spacer + Sml/Med Mask                              22731   

                                                    ratio of items  
post_code bnf_name                                                  
B11 4BW   3m Health Care_Cavilon Durable Barrier C        0.000308  
          3m Health Care_Cavilon No Sting Barrier         0.000088  
          Abasaglar KwikPen_100u/ml 3ml Pf Pen            0.000088  
          Abidec_Dps                                      0.002772  
          Able Spacer + Sml/Med Mask                      0.000044

In [35]:
ranged_df = sum_items_df.sort_values(['post_code', 'ratio of items', 'bnf_name'], ascending = [True, False, True])

In [36]:
ranged_df.head()

items  total per post code  \
post_code bnf_name                                                         
B11 4BW   Salbutamol_Inha 100mcg (200 D) CFF    706                22731   
          Paracet_Tab 500mg                     451                22731   
          Metformin HCl_Tab 500mg               387                22731   
          Lansoprazole_Cap 30mg (E/C Gran)      385                22731   
          Amoxicillin_Cap 500mg                 350                22731   

                                              ratio of items  
post_code bnf_name                                            
B11 4BW   Salbutamol_Inha 100mcg (200 D) CFF        0.031059  
          Paracet_Tab 500mg                         0.019841  
          Metformin HCl_Tab 500mg                   0.017025  
          Lansoprazole_Cap 30mg (E/C Gran)          0.016937  
          Amoxicillin_Cap 500mg                     0.015397

In [37]:
# reset index for 'ranged_df'
ranged_df_rs = ranged_df.reset_index()

In [38]:
ranged_df_rs.head()

,post_code,bnf_name,items,total per post code,ratio of items
0,B11 4BW,Salbutamol_Inha 100mcg (200 D) CFF,706,22731,0.031059
1,B11 4BW,Paracet_Tab 500mg,451,22731,0.019841
2,B11 4BW,Metformin HCl_Tab 500mg,387,22731,0.017025
3,B11 4BW,Lansoprazole_Cap 30mg (E/C Gran),385,22731,0.016937
4,B11 4BW,Amoxicillin_Cap 500mg,350,22731,0.015397


In [39]:
# number of postal codes
len(set(ranged_df_rs['post_code']))

289

In [40]:
# set of all postal codes
s = set(ranged_df_rs['post_code'])

In [41]:
list_s = ['B11 4BW',
 'B12 9LP',
 'B18 7AL',
 'B21 9RY',
 'B23 6DJ',
 'B26 1TH',
 'B61 0AZ',
 'B70 7AW',
 'B72 1RL',
 'B8 1RZ',
 'B9 5PU',
 'B90 3LX',
 'BA5 1XJ',
 'BB11 2DL',
 'BB12 8BS',
 'BB2 1AX',
 'BB3 1PY',
 'BB4 5SL',
 'BB4 7PL',
 'BB7 2JG',
 'BB8 0JZ',
 'BB9 7SR',
 'BD16 4RP',
 'BD19 5AP',
 'BD3 8QH',
 'BD4 7SS',
 'BH14 0DJ',
 'BH14 9ET',
 'BH18 8EE',
 'BH23 3AF',
 'BL1 3RG',
 'BL1 8TU',
 'BL2 6NT',
 'BL3 5HP',
 'BL9 0NJ',
 'BL9 0SN',
 'BN1 6AG',
 'BN1 8DD',
 'BN9 9PW',
 'BR2 9GT',
 'BR3 3FD',
 'BS16 3TD',
 'BS23 3HQ',
 'BS4 1WH',
 'BS4 4HU',
 'BS48 2XX',
 'BS48 3HA',
 'CA11 8HW',
 'CB22 3HU',
 'CB9 8HF',
 'CH1 4DS',
 'CH41 8DB',
 'CH44 5UF',
 'CH62 5HS',
 'CH62 6EE',
 'CH65 6TG',
 'CH66 3PB',
 'CM18 6LY',
 'CR0 0JA',
 'CT11 8AD',
 'CV1 4FS',
 'CV12 8NQ',
 'CV21 2DN',
 'CV6 2FL',
 'CV6 6DR',
 'CW1 3AW',
 'CW5 5NX',
 'CW7 1AT',
 'DA1 2HA',
 'DA11 8BZ',
 'DN16 2AB',
 'DN22 7XF',
 'DN31 3AE',
 'DN34 4GB',
 'DN36 4QG',
 'DN6 0HZ',
 'DN8 4BQ',
 'DY11 6SF',
 'E15 4ES',
 'E7 0EP',
 'FY2 0JG',
 'FY4 1TJ',
 'FY5 2TZ',
 'FY5 3LF',
 'FY7 8GU',
 'FY8 5DZ',
 'GL1 3PX',
 'GL20 5GJ',
 'GL20 5QQ',
 'GL20 5RY',
 'GL50 4DP',
 'GU9 9QS',
 'HA0 4UZ',
 'HA3 7LT',
 'HD6 1AT',
 'HG1 5AR',
 'HR1 2JB',
 'HR6 8HD',
 'HU7 4DW',
 'HU9 2LJ',
 'IG7 4DF',
 'IP22 4WG',
 'KT12 3LB',
 'KT14 6DH',
 'KT16 8HZ',
 'KT6 6EZ',
 'KT6 7QU',
 'L31 0DJ',
 'L31 8BP',
 'L31 8BR',
 'L34 1ND',
 'L36 0UB',
 'L36 5TH',
 'L36 7XY',
 'L5 0QW',
 'L7 6HD',
 'L8 6QP',
 'L9 1AD',
 'LA1 1PN',
 'LA1 2LG',
 'LA1 4JS',
 'LE10 1DS',
 'LE18 2EW',
 'LE2 9BU',
 'LE5 3GH',
 'LE7 2EQ',
 'LN1 2NU',
 'LN2 2JP',
 'LN6 0QQ',
 'LN7 6NX',
 'LS9 9EF',
 'M11 4EJ',
 'M13 9UJ',
 'M22 5RX',
 'M22 9UH',
 'M26 2SP',
 'M28 3AT',
 'M30 0NU',
 'M34 2AJ',
 'M34 3RA',
 'M35 0AD',
 'M7 1RD',
 'ME12 1UP',
 'ME12 3LT',
 'ME8 8AA',
 'N12 9SS',
 'N15 4JR',
 'N9 0TW',
 'N9 7HD',
 'NE10 9QG',
 'NE20 9SD',
 'NE24 1DX',
 'NE24 1HD',
 'NE27 0HJ',
 'NE29 0SF',
 'NE31 1NU',
 'NE33 4JP',
 'NE37 2PU',
 'NE38 7NQ',
 'NE38 8JF',
 'NG10 1RY',
 'NG2 7SD',
 'NG6 8QJ',
 'NG7 3GW',
 'NG7 5HY',
 'NG8 6AS',
 'NN16 8DN',
 'NN3 8DW',
 'NR28 0BQ',
 'NW10 8RY',
 'OL1 1NL',
 'OL11 1DN',
 'OL2 6QW',
 'OL4 1YN',
 'OL6 6HD',
 'OL6 7SR',
 'OL9 0LH',
 'OL9 7AY',
 'OX12 9BN',
 'OX16 9AD',
 'PE1 2QP',
 'PE21 8EG',
 'PL7 1AD',
 'PO11 9AP',
 'PR25 1HR',
 'PR3 1PB',
 'RM3 9SU',
 'S6 4JQ',
 'S63 9EH',
 'S65 1DA',
 'S70 1QE',
 'S74 9AF',
 'SE1 6JP',
 'SE15 5LJ',
 'SE15 6DB',
 'SE17 3BD',
 'SE25 5NT',
 'SE8 4BG',
 'SG8 8HY',
 'SK11 6JL',
 'SK6 1ND',
 'SK8 3JD',
 'SK8 6LU',
 'SL9 9SA',
 'SM3 8EP',
 'SM6 0HY',
 'SR4 7XF',
 'SR4 9AS',
 'SR5 2LT',
 'SR7 7JE',
 'SR8 2RT',
 'SS0 7AF',
 'SS13 3HQ',
 'SS4 1AY',
 'SS7 4EA',
 'SS8 0JA',
 'SS9 5UU',
 'ST1 4PB',
 'ST3 1EQ',
 'ST3 5DQ',
 'ST3 6AB',
 'ST7 2LU',
 'ST7 4AY',
 'ST8 6AG',
 'TF3 2JZ',
 'TF4 2LL',
 'TN24 0GP',
 'TN34 1BA',
 'TQ2 6HW',
 'TQ2 8JG',
 'TR1 2JA',
 'TR1 3ER',
 'TS1 2NX',
 'TS10 4NW',
 'TS14 7DJ',
 'TS17 0EE',
 'TS23 2DG',
 'TS24 7PW',
 'TS3 6AL',
 'TS6 6TD',
 'TS8 0TL',
 'TW13 4GU',
 'TW3 2DY',
 'TW3 3LN',
 'TW5 9ER',
 'TW8 8DS',
 'UB5 6WL',
 'W10 6DZ',
 'W12 7FG',
 'W4 1RX',
 'WA1 3RB',
 'WA10 2DJ',
 'WA11 0JN',
 'WA11 0NA',
 'WA15 6PH',
 'WA3 3GS',
 'WA6 6RX',
 'WA7 1AB',
 'WA7 2UT',
 'WA9 1LN',
 'WD18 0JP',
 'WD18 7QR',
 'WD23 2NN',
 'WD3 7DJ',
 'WF13 1HN',
 'WF16 0HH',
 'WF5 8DF',
 'WF9 3AP',
 'WN2 5NG',
 'WN3 5HL',
 'WN5 9QX',
 'WN7 1HR',
 'WN7 2PE',
 'WR11 4BS',
 'WR14 2GP',
 'WR9 8RD',
 'WS1 3PS',
 'WS10 8SY',
 'WS11 9SE',
 'WS13 6JL',
 'WS13 7HT',
 'WS2 0BA',
 'WS3 1ET',
 'WS3 3JP',
 'WS7 0EW',
 'WS9 8AJ',
 'WV13 2DR',
 'YO11 1UB',
 'YO16 4LZ']

In [42]:
ranged_df_rs_new = ranged_df_rs[['post_code', 'bnf_name', 'ratio of items']]
print(ranged_df_rs_new.shape)
ranged_df_rs_new.head()

(612859, 3)


,post_code,bnf_name,ratio of items
0,B11 4BW,Salbutamol_Inha 100mcg (200 D) CFF,0.031059
1,B11 4BW,Paracet_Tab 500mg,0.019841
2,B11 4BW,Metformin HCl_Tab 500mg,0.017025
3,B11 4BW,Lansoprazole_Cap 30mg (E/C Gran),0.016937
4,B11 4BW,Amoxicillin_Cap 500mg,0.015397


In [43]:
items_by_region = []
for i in np.arange(0,100):
    sub_df = ranged_df_rs_new[ranged_df_rs_new['post_code']==list_s[i]]
    items_by_region.append(tuple(sub_df.iloc[0]))

In [44]:
print(len(items_by_region))
#print(items_by_region)

100


In [45]:
#items_by_region = [("B11 4BW", "Salbutamol_Inha 100mcg (200 D) CFF", 0.0310589063)] * 100

In [46]:
grader.score.dw__items_by_region(items_by_region) 

Your score:  0.8600000000000005


## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [47]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [48]:
opioids_join = '|'.join(opioids)
opioids_join

'morphine|oxycodone|methadone|fentanyl|pethidine|buprenorphine|propoxyphene|codeine'

In [49]:
opioid_codes = chem.loc[chem['NAME'].str.contains(opioids_join, case=False)]['CHEM SUB'].tolist()
len(opioid_codes)

35

In [50]:
chem_new = chem
chem_new.columns = ['bnf_code','chem_name']
chem_new.head()

,bnf_code,chem_name
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


In [51]:
scripts.head()

,code,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [52]:
scripts['opioid_prescription'] = scripts['bnf_code'].isin(opioid_codes)

In [53]:
scripts = scripts.rename({'code':'practice'}, axis = 1)

In [54]:
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,opioid_prescription
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,False
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,False
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,False
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,False
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,False


In [55]:
set(scripts.opioid_prescription)

{False, True}

In [56]:
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
1,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
2,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
3,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ
4,A81006,TENNANT STREET MEDICAL PRACTICE,TENNANT ST MED PRACT,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AT


In [57]:
pract = practices[['code', 'name']]
pract.columns = ['practice', 'name']
pract.head()

,practice,name
0,A81002,QUEENS PARK MEDICAL CENTRE
1,A81003,VICTORIA MEDICAL PRACTICE
2,A81004,WOODLANDS ROAD SURGERY
3,A81005,SPRINGWOOD SURGERY
4,A81006,TENNANT STREET MEDICAL PRACTICE


In [58]:
opioids_per_practice = scripts.groupby('practice')['opioid_prescription'].mean().rename('frac')
opioids_per_practice.head()

practice
A81005    0.033179
A81007    0.043329
A81011    0.046556
A81012    0.042793
A81017    0.038140
Name: frac, dtype: float64

In [59]:
opioids_per_practice_std = scripts.groupby('practice')['opioid_prescription'].std().rename('frac_std')
opioids_per_practice_std.head()

practice
A81005    0.179162
A81007    0.203666
A81011    0.210753
A81012    0.202466
A81017    0.191578
Name: frac_std, dtype: float64

In [60]:
overall_rate = scripts['opioid_prescription'].mean()
overall_rate

0.03580276471367961

In [61]:
overall_rate_std = scripts['opioid_prescription'].std()
overall_rate_std

0.18579817605238425

In [62]:
relative_opioids_per_practice = (opioids_per_practice - overall_rate).rename('relative')
relative_opioids_per_practice.head()

practice
A81005   -0.002624
A81007    0.007526
A81011    0.010753
A81012    0.006990
A81017    0.002337
Name: relative, dtype: float64

In [63]:
opioid = scripts.groupby('practice')['opioid_prescription'].sum().rename('opioid')
opioid.head()

practice
A81005    50.0
A81007    63.0
A81011    73.0
A81012    57.0
A81017    82.0
Name: opioid, dtype: float64

In [64]:
total = scripts.groupby('practice')['bnf_code'].count().rename('total')
total.head()

practice
A81005    1507
A81007    1454
A81011    1568
A81012    1332
A81017    2150
Name: total, dtype: int64

In [65]:
standard_error_per_practice = (overall_rate_std/(total**0.5)).rename('std_err')
standard_error_per_practice.head()

practice
A81005    0.004786
A81007    0.004873
A81011    0.004692
A81012    0.005091
A81017    0.004007
Name: std_err, dtype: float64

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [66]:
opioid_scores = (relative_opioids_per_practice/standard_error_per_practice).rename('opioid_scores')
opioid_scores.head()

practice
A81005   -0.548306
A81007    1.544557
A81011    2.291795
A81012    1.373060
A81017    0.583168
Name: opioid_scores, dtype: float64

In [67]:
merged = pd.concat([opioid, total, opioids_per_practice, relative_opioids_per_practice, standard_error_per_practice,opioid_scores], axis=1)

In [68]:
merged = merged.reset_index()

In [69]:
merged.head()

,practice,opioid,total,frac,relative,std_err,opioid_scores
0,A81005,50.0,1507,0.033179,-0.002624,0.004786,-0.548306
1,A81007,63.0,1454,0.043329,0.007526,0.004873,1.544557
2,A81011,73.0,1568,0.046556,0.010753,0.004692,2.291795
3,A81012,57.0,1332,0.042793,0.006990,0.005091,1.373060
4,A81017,82.0,2150,0.038140,0.002337,0.004007,0.583168


In [70]:
final_df = merged.merge(pract, on='practice', how='left')

In [71]:
final_df.head()

,practice,opioid,total,frac,relative,std_err,opioid_scores,name
0,A81005,50.0,1507,0.033179,-0.002624,0.004786,-0.548306,SPRINGWOOD SURGERY
1,A81007,63.0,1454,0.043329,0.007526,0.004873,1.544557,BANKHOUSE SURGERY
2,A81011,73.0,1568,0.046556,0.010753,0.004692,2.291795,CHADWICK PRACTICE
3,A81012,57.0,1332,0.042793,0.006990,0.005091,1.373060,WESTBOURNE MEDICAL CENTRE
4,A81017,82.0,2150,0.038140,0.002337,0.004007,0.583168,WOODBRIDGE PRACTICE


In [72]:
final_df.sort_values('opioid_scores', ascending = False , inplace=True)

In [73]:
final_df.head()

,practice,opioid,total,frac,relative,std_err,opioid_scores,name
829,Y01852,6.0,7,0.857143,0.821340,0.070225,11.695818,NATIONAL ENHANCED SERVICE
874,Y03006,2.0,2,1.000000,0.964197,0.131379,7.339043,OUTREACH SERVICE NH / RH
895,Y03668,11.0,60,0.183333,0.147531,0.023986,6.150582,BRISDOC HEALTHCARE SERVICES OOH
296,G81703,7.0,36,0.194444,0.158642,0.030966,5.123032,H&R P C SPECIAL SCHEME
948,Y04997,28.0,321,0.087227,0.051425,0.010370,4.958866,HMR BARDOC OOH


In [74]:
final = final_df.drop_duplicates('name')
final.head()

,practice,opioid,total,frac,relative,std_err,opioid_scores,name
829,Y01852,6.0,7,0.857143,0.821340,0.070225,11.695818,NATIONAL ENHANCED SERVICE
874,Y03006,2.0,2,1.000000,0.964197,0.131379,7.339043,OUTREACH SERVICE NH / RH
895,Y03668,11.0,60,0.183333,0.147531,0.023986,6.150582,BRISDOC HEALTHCARE SERVICES OOH
296,G81703,7.0,36,0.194444,0.158642,0.030966,5.123032,H&R P C SPECIAL SCHEME
948,Y04997,28.0,321,0.087227,0.051425,0.010370,4.958866,HMR BARDOC OOH


In [75]:
result = final[['name','opioid_scores','total']]
result.head()

,name,opioid_scores,total
829,NATIONAL ENHANCED SERVICE,11.695818,7
874,OUTREACH SERVICE NH / RH,7.339043,2
895,BRISDOC HEALTHCARE SERVICES OOH,6.150582,60
296,H&R P C SPECIAL SCHEME,5.123032,36
948,HMR BARDOC OOH,4.958866,321


In [76]:
anomalies = result.head(100)

In [77]:
anomalies = anomalies.values.tolist()

In [78]:
anomalies

[['NATIONAL ENHANCED SERVICE', 11.695817862936027, 7],
 ['OUTREACH SERVICE NH / RH', 7.339043019238823, 2],
 ['BRISDOC HEALTHCARE SERVICES OOH', 6.1505817490838295, 60],
 ['H&R P C SPECIAL SCHEME', 5.123032414033079, 36],
 ['HMR BARDOC OOH', 4.958866438487605, 321],
 ['INTEGRATED CARE 24 LTD (CWSX OOH)', 4.8888781604828235, 426],
 ['DARWEN HEALTHCARE', 4.8391589686363385, 1917],
 ['THE LIMES MEDICAL PRACTICE', 4.546841872334426, 1321],
 ['IC24 LTD (BRIGHTON & HOVE OOH)', 4.335047010605197, 357],
 ['OLDHAM 7 DAY ACCESS HUB2 OOH', 4.31178403661019, 56],
 ['IC24 LTD (NORFOLK & WISBECH OOH)', 4.2575005924727645, 489],
 ['ROSSENDALE MIU & OOH', 4.256827446322491, 18],
 ['BURY WALK-IN CENTRE', 4.150589122881536, 138],
 ['IC24 LTD (HORSHAM & MID SUSSEX OOH)', 3.7816207038443523, 215],
 ['LCW HOUNSLOW CCG OOH', 3.582848546206269, 69],
 ['WEEKEND WORKING EASINGTON NORTH', 3.565938771129764, 278],
 ['COMPASS ENFIELD', 3.5587202651067935, 7],
 ['BASSETLAW DRUG & ALCOHOL SERVICE', 3.53326410251154

In [79]:
grader.score.dw__script_anomalies(anomalies)

Error!
You have been rate limited for exceeding the limit of 3 per 1 minute.
Please slow down your submission rate.


## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [80]:
scripts16 = pd.read_csv('./dw-data/201606scripts_sample.csv.gz')

In [81]:
scripts16.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85638,0301011R0,Salamol_Inha 100mcg (200 D) CFF (Teva),2,2.92,2.73,2
1,N85638,0301011R0,Easyhaler_Salbutamol Sulf 200mcg (200D),1,6.63,6.15,1
2,N85638,0301020I0,Ipratrop Brom_Inh Soln 500mcg/2ml Ud,1,1.77,1.75,12
3,N85638,0301020I0,Ipratrop Brom_Inh Soln 250mcg/1ml Ud,1,4.47,4.15,20
4,N85638,0302000C0,Clenil Modulite_Inha 50mcg (200D),1,3.70,3.44,1


In [82]:
print(scripts16.shape)

(973193, 7)


In [83]:
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,opioid_prescription
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,False
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,False
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,False
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,False
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,False


In [84]:
# calculate the growth rate
growth = (scripts.bnf_name.value_counts() - scripts16.bnf_name.value_counts())/scripts16.bnf_name.value_counts()

In [85]:
growth

365 Film 10cm x 12cm VP Adh Film Dress           NaN
365 Film 15cm x 20cm VP Adh Film Dress           NaN
365 Film 4cm x 5cm VP Adh Film Dress             NaN
365 Non Adherent 10cm x 10cm Pfa Plas Fa    0.000000
365 Non Adherent 10cm x 20cm Pfa Plas Fa         NaN
                                              ...   
nSpire PiKo-1 Stnd Range Peak Flow Meter   -0.500000
nSpire Pocket Peak Low Range Peak Flow M    1.000000
nSpire Pocket Peak Stnd Range Peak Flow     1.500000
oraNurse_Toothpaste Orig (1450ppm)          1.000000
palmdoc (Reagent)_Strips                    0.666667
Name: bnf_name, Length: 15424, dtype: float64

In [86]:
df = pd.DataFrame(dict(growth = growth, item16 = scripts16.bnf_name.value_counts())).reset_index()

In [87]:
df.head()

,index,growth,item16
0,365 Film 10cm x 12cm VP Adh Film Dress,NaN,NaN
1,365 Film 15cm x 20cm VP Adh Film Dress,NaN,1.0
2,365 Film 4cm x 5cm VP Adh Film Dress,NaN,1.0
3,365 Non Adherent 10cm x 10cm Pfa Plas Fa,0.0,3.0
4,365 Non Adherent 10cm x 20cm Pfa Plas Fa,NaN,NaN


In [88]:
df.fillna(0, inplace = True)

In [89]:
df.head()

,index,growth,item16
0,365 Film 10cm x 12cm VP Adh Film Dress,0.0,0.0
1,365 Film 15cm x 20cm VP Adh Film Dress,0.0,1.0
2,365 Film 4cm x 5cm VP Adh Film Dress,0.0,1.0
3,365 Non Adherent 10cm x 10cm Pfa Plas Fa,0.0,3.0
4,365 Non Adherent 10cm x 20cm Pfa Plas Fa,0.0,0.0


In [90]:
# remove the elements with less than 50 prescriptions
df = df[df['item16'] >= 50]

In [91]:
df.shape

(3519, 3)

In [92]:
# sort the dataframe by the descending order of growth rate
df = df.sort_values(by = 'growth', ascending = False)

In [93]:
df.head()

,index,growth,item16
1985,Butec_Transdermal Patch 5mcg/hr,3.467742,62.0
1983,Butec_Transdermal Patch 10mcg/hr,3.000000,69.0
5742,Fostair NEXThaler_Inh 200mcg/6mcg (120D),1.430233,86.0
11270,Pneumococcal_Vac 0.5ml Vl (23 Valent),1.269430,193.0
13357,Spiolto Respimat_Inha2.5/2.5mcg(60D)+Dev,1.269231,52.0


In [94]:
# pick the top 50 and the last 50 elements:
script_growth = list(pd.concat([df.head(50), df.tail(50)]).itertuples(index = False, name = None))

In [95]:
script_growth

[('Butec_Transdermal Patch 5mcg/hr', 3.467741935483871, 62.0),
 ('Butec_Transdermal Patch 10mcg/hr', 3.0, 69.0),
 ('Fostair NEXThaler_Inh 200mcg/6mcg (120D)', 1.430232558139535, 86.0),
 ('Pneumococcal_Vac 0.5ml Vl (23 Valent)', 1.2694300518134716, 193.0),
 ('Spiolto Respimat_Inha2.5/2.5mcg(60D)+Dev', 1.2692307692307692, 52.0),
 ('Trulicity_Inj 1.5mg/0.5ml Pf Pen', 1.1851851851851851, 54.0),
 ('CosmoCol_Paed Oral Pdr Sach 6.9g', 1.1774193548387097, 62.0),
 ('Dulaglutide_Inj 1.5mg/0.5ml Pf Dev', 0.9634146341463414, 82.0),
 ('ViATIM_Vac D/Chamber 160u/25mcg 1ml Pfs', 0.912, 125.0),
 ('Empagliflozin_Tab 25mg', 0.896, 125.0),
 ('CareSens Lancets 0.31mm/30 Gauge', 0.8955223880597015, 67.0),
 ('Fostair_Inh 200mcg/6mcg (120D) CFF', 0.8616600790513834, 253.0),
 ('Orbis Nor Saline Sod Chlor 0.9% Nsl Dps', 0.8588235294117647, 85.0),
 ('Umeclidinium Brom_Inh 65mcg (30D)', 0.8275862068965517, 87.0),
 ('Ultibro Breezhaler_Pdr Inh Cap + Dev', 0.8016528925619835, 121.0),
 ('Medihoney_Barrier Crm', 0.7

In [96]:
#script_growth = [("Butec_Transdermal Patch 5mcg\/hr", 3.4677419355, 62.0)] * 100

In [97]:
grader.score.dw__script_growth(script_growth)

Error!
You have been rate limited for exceeding the limit of 3 per 1 minute.
Please slow down your submission rate.


## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [98]:
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
1,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
2,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
3,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ
4,A81006,TENNANT STREET MEDICAL PRACTICE,TENNANT ST MED PRACT,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AT


In [99]:
practices.shape

(12019, 7)

In [100]:
practices_unique = practices.sort_values(['code', 'post_code']).drop_duplicates(subset = 'code')

In [101]:
group_by_bnf_code = scripts.groupby(['bnf_code'])['bnf_code'].count().reset_index(name = 'bnf_code_count')

In [102]:
group_by_bnf_code['p'] = group_by_bnf_code['bnf_code_count']/len(scripts)
px = 1/len(group_by_bnf_code)
print(px)

0.0005063291139240507


In [103]:
group_by_bnf_code['rare'] = np.where(group_by_bnf_code['p'] < 0.1*px, True, False)

In [104]:
scripts_rare = pd.merge(scripts, group_by_bnf_code, how = 'inner', on = 'bnf_code')

In [114]:
scripts_rare.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,opioid_prescription,bnf_code_count,p,rare
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,False,1035,0.001064,False
1,N81013,0106020C0,Bisacodyl_Tab E/C 5mg,19,28.10,28.14,860,False,1035,0.001064,False
2,N81029,0106020C0,Bisacodyl_Tab E/C 5mg,50,67.73,67.51,2074,False,1035,0.001064,False
3,N81029,0106020C0,Bisacodyl_Suppos 10mg,1,1.77,1.75,6,False,1035,0.001064,False
4,N81029,0106020C0,Dulcolax_Tab 5mg,1,3.78,3.61,56,False,1035,0.001064,False


In [115]:
total_cost_by_practice = scripts_rare.groupby('practice')['act_cost'].sum().reset_index(name = 'total_cost')

In [116]:
rare_cost_by_practice = scripts_rare[scripts_rare['rare'] == True].groupby('practice')['act_cost'].sum().reset_index(name = 'rare_cost')

In [119]:
rare_cost_by_practice.head()

,practice,rare_cost
0,A81005,1247.83
1,A81007,951.06
2,A81011,816.02
3,A81012,1145.11
4,A81017,1712.15


In [121]:
script_cost_by_practice = pd.merge(total_cost_by_practice, rare_cost_by_practice, on = 'practice', how = 'left')

In [124]:
script_cost_by_practice.fillna(0, inplace = True)

In [126]:
script_cost_by_practice['rare_cost_prop'] = script_cost_by_practice['rare_cost']/script_cost_by_practice['total_cost']

In [128]:
script_cost_by_practice.head()

,practice,total_cost,rare_cost,rare_cost_prop
0,A81005,103840.82,1247.83,0.012017
1,A81007,113482.49,951.06,0.008381
2,A81011,159507.03,816.02,0.005116
3,A81012,83296.81,1145.11,0.013747
4,A81017,232656.17,1712.15,0.007359


In [130]:
total_cost = scripts_rare['act_cost'].sum()
total_cost

66164096.12000001

In [132]:
total_rare_cost = scripts_rare[scripts_rare['rare'] == True]['act_cost'].sum()

In [133]:
overall_rare_cost = total_rare_cost/total_cost

In [135]:
script_cost_by_practice['relative_rare_cost_prop'] = script_cost_by_practice['rare_cost_prop'] - overall_rare_cost

In [137]:
script_cost_by_practice['standard_errors'] = script_cost_by_practice['relative_rare_cost_prop'].std()

In [139]:
script_cost_by_practice['z-score'] = script_cost_by_practice['relative_rare_cost_prop']/script_cost_by_practice['standard_errors']

In [140]:
print(script_cost_by_practice.shape)
script_cost_by_practice.head()

(856, 7)


,practice,total_cost,rare_cost,rare_cost_prop,relative_rare_cost_prop,standard_errors,z-score
0,A81005,103840.82,1247.83,0.012017,-0.003946,0.060509,-0.065216
1,A81007,113482.49,951.06,0.008381,-0.007582,0.060509,-0.125308
2,A81011,159507.03,816.02,0.005116,-0.010847,0.060509,-0.179263
3,A81012,83296.81,1145.11,0.013747,-0.002216,0.060509,-0.036615
4,A81017,232656.17,1712.15,0.007359,-0.008604,0.060509,-0.142190


In [142]:
practices_unique = practices.sort_values('name').groupby('code', sort = False).first()

In [144]:
practices_unique.reset_index(inplace = True)

In [145]:
merge_scripts_practice_z_scores = script_cost_by_practice.merge(practices_unique[['name', 'code']], how = 'inner', left_on = 'practice', right_on = 'code', sort = False)

In [148]:
merge_scripts_practice_z_scores.head()

,practice,total_cost,rare_cost,rare_cost_prop,relative_rare_cost_prop,standard_errors,z-score,name,code
0,A81005,103840.82,1247.83,0.012017,-0.003946,0.060509,-0.065216,SPRINGWOOD SURGERY,A81005
1,A81007,113482.49,951.06,0.008381,-0.007582,0.060509,-0.125308,BANKHOUSE SURGERY,A81007
2,A81011,159507.03,816.02,0.005116,-0.010847,0.060509,-0.179263,CHADWICK PRACTICE,A81011
3,A81012,83296.81,1145.11,0.013747,-0.002216,0.060509,-0.036615,WESTBOURNE MEDICAL CENTRE,A81012
4,A81017,232656.17,1712.15,0.007359,-0.008604,0.060509,-0.142190,WOODBRIDGE PRACTICE,A81017


In [151]:
print(merge_scripts_practice_z_scores.shape)

(856, 9)


In [152]:
merge_scripts_practice_z_scores = merge_scripts_practice_z_scores.sort_values('name', ascending = True)

In [154]:
merge_scripts_practice_z_scores.drop_duplicates(subset = ['code', 'name'], inplace = True)

In [156]:
merge_scripts_practice_z_scores.head()

,practice,total_cost,rare_cost,rare_cost_prop,relative_rare_cost_prop,standard_errors,z-score,name,code
629,P87652,41866.90,271.21,0.006478,-0.009485,0.060509,-0.156754,1/LOWER BROUGHTON MEDICAL PRACTICE,P87652
627,P87020,70409.18,291.06,0.004134,-0.011829,0.060509,-0.195493,2/ST ANDREWS MEDICAL PRACTICE,P87020
630,P87654,73060.62,208.44,0.002853,-0.013110,0.060509,-0.216661,3/LOWER BROUGHTON MEDICAL PRACTICE,P87654
631,P87659,41531.09,667.62,0.016075,0.000112,0.060509,0.001856,3/ST ANDREWS MEDICAL CENTRE,P87659
628,P87036,29124.57,311.13,0.010683,-0.005280,0.060509,-0.087263,4/LOWER BROUGHTON MEDICAL PRACTICE,P87036


In [159]:
rare_scripts = []

for indew, row in merge_scripts_practice_z_scores.iterrows():
    rare_scripts.append((row['practice'], row['name'], row['z-score']))
    
rare_scripts = sorted(rare_scripts, key = lambda x : x[2], reverse = True)[:100]   

In [160]:
rare_scripts

[('Y03472', 'CONSULTANT DIABETES TEAM', 16.262687124655073),
 ('Y05320', 'DMC COMMUNITY DERMATOLOGY RBWF', 15.128648195416869),
 ('Y04404', 'OUTPATIENTS JUBILEE HEALTH CENTRE', 7.542139356104622),
 ('Y03484', 'DMC COMMUNITY DERMATOLOGY CLINIC', 7.2872222002978235),
 ('Y04424', 'DMC HEALTHCARE', 6.8386141814328685),
 ('Y00631', 'BINGLEY DERMATOLOGY CLINIC', 5.75599754983774),
 ('A89038', 'BARMSTON MEDICAL CENTRE', 5.664940368539937),
 ('Y01696', 'BASSETLAW HOSPICE OF THE GOOD SHEPHERD', 4.29068517147492),
 ('Y03699', 'OLDHAM DERMATOLOGY SERVICE', 4.103743999157243),
 ('Y02045', 'VERNOVA HEALTHCARE CIC', 3.1807154704143983),
 ('Y02823', 'DMC VICARAGE LANE', 3.054121649498702),
 ('Y00997', 'COMMUNITY DERMATOLOGY SERVICE', 3.0181283595352766),
 ('Y05019', 'OLDHAM TOTAL SKIN SERVICE', 2.7247515958989283),
 ('Y01003', 'BURY OOH', 2.350436347139857),
 ('Y05431', 'NOTTS APPLIANCE MANAGEMENT SERVICE', 2.037158543233387),
 ('Y03557', 'IPU 10-17 RECOVERY COVENTRY', 2.0304495001187792),
 ('Y05419'

In [161]:
grader.score.dw__rare_scripts(rare_scripts)

Your score:  1.0


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*